# Energy minimization in mBuild using OpenBabel constraints

This notebook provides an overview of the syntax to define various constraints for mBuilds wrapper to OpenBabel.  The features discussed here include:

- Specifying `Compounds` whose position will stay fixed during energy minimization
- Defining distance constraints between particle pairs
- Specifying `Compounds` whose interactions are to be ignored during energy minimization


In [1]:
import mbuild as mb
import numpy as np
import warnings
warnings.filterwarnings('ignore') 


from mbuild.lib.recipes.alkane import Alkane

## Fixing Compound positions

OpenBabel allows a user to specify a list of atoms whose position will remained "fixed" during the energy minimization. The wrapper in mBuild allows users to specify `Compounds` to be fixed via the fixed_compounds argument. 

In the code below, we will specify the first methyl group to be fixed during the optimization.  Note, mBuild will automatically traverse the hierarchy such that it will fix all particles contained within a `Compound` at any level below it; in this case one  C atom and three H atom. 

This example is similar to the anchor_point example in the Basic Energy Minimization notebook. 

In [2]:
hexane = Alkane(n=6)
fixed = hexane.labels['chain'].labels['CH3'][0]
initial_com = fixed.pos

hexane.energy_minimize(shift_com=False, fixed_compounds=fixed)
final_com = fixed.pos


diff = np.linalg.norm(initial_com-final_com)
print("Net change in anchor COM: {0:.10f} nm ".format(diff))

Net change in anchor COM: 0.0013435056 nm 


If we run the code cell above, we will see a net change in the COM is on the order of 1e-3 nm, which might make you think the position has not been held fixed. However, in OpenBabel, fixed compounds are not truly fixed, but rather restrained by a strong harmonic spring to their initial position, and thus small changes in position are to be expected during the optimization. 

To better demonstrate that positions are reasonably held constant during the optimization, let us expand this example such that we fix the first half the molecule. Visualization of this structure will show that the initial planar structure is retained in the first two monomers, while the rest of the chain is adjusted.

In [3]:
hexane = Alkane(n=6)
fixed1 = hexane.labels['chain'].labels['CH3'][0]
fixed2 = hexane.labels['chain'].labels['monomer'][0]
fixed3 = hexane.labels['chain'].labels['monomer'][1]

hexane.energy_minimize(shift_com=False, 
                       fixed_compounds=[fixed1, fixed2, fixed3])

hexane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Unless otherwise specified, a fixed `Compound` will have position fixed in x,y and z directions.  To specify which directions to restrain the positions, we can use the following format: 

`[Compound, (x bool value, y bool value, z bool value)]`.  Note, bool values can either be a tuple or list with no impact on the code itself; here, a tuple is used for visual clarity. 

In the code below, we only fix the z-position of the terminal methyl groups:

In [4]:
hexane = Alkane(n=6)
fixed1 = hexane.labels['chain'].labels['CH3'][0]
fixed2 = hexane.labels['chain'].labels['CH3'][1]


hexane.energy_minimize(shift_com=False, 
                       fixed_compounds=[
                                        [fixed1, (False, False, True)],
                                        [fixed2, (False, False, True)]
                                       ])

## Defining distance constraints

OpenBabel allows constraints to be defined between pairs of atoms.  To specify these with mBuild, we will pass a list of `Compound` pairs and associated distances via the `distance_constraints` argument. Note that, the pairs of `Compounds` must represents individual particles, rather than collections of particles. That is, the code will not let you specify a distance constraint between the two terminal methyl groups, but will allow you to specify a distance constraint between the C atoms in the methyl group. 

The format of each distance constraint is `[(Compound1, Compound2), distance]`; again, the use of a tuple as a opposed to a list does not impact any function of the code itself, and pairs could be alternatively be defined as a list. Note, distance units are defined in nm, consistent with mBuild.

The following code fixes the distance between the terminal C atoms in a hexane molecule, where we specify a small separation to enforce a compacted design.  Note, any number of distance constraints can be specified. 


In [5]:
hexane = Alkane(n=6)
point0 = hexane.labels['chain'].labels['CH3'][0].labels['C'][0]
point1 = hexane.labels['chain'].labels['CH3'][1].labels['C'][0]

hexane.energy_minimize(steps=10000,shift_com=False, distance_constraints=[ [(point0, point1), 0.5 ]])

distance = np.linalg.norm(point0.pos-point1.pos)

print("distance between two restrained carbons: ", distance)

distance between two restrained carbons:  0.50015455


In [6]:
hexane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Specificying compounds to ignore

We can specify `Compounds` to ignore as well.  By ignore, we mean that OpenBabel won't attempt to move the particles in the `Compound` and it will not apply any force field parameters automatically to the particles. 

This is particularly useful if we want to use a `Compound` as and anchoring point, e.g., if reverse-mapping a coarse-grained particle, we might want to energy minimize the atomistic atoms ensuring they maintain the target distance to the associated CG interaction site. 

Note, we need to assign an element to any `Compound` that we wish to ignore (or have the name represent an element) to be able to perform the energy minimization. 

Let us add an arbitrary Helium in the middle of the chain (overlapping a Carbon atom), slightly offset so it can be easily visualized.

In [7]:
hexane = Alkane(n=6)

helium = mb.Compound(pos=hexane.labels['chain'].labels['monomer'][2].pos+[0.0,0.0,0.05], name='He', element='He')
hexane.add(helium)

hexane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

If we run the energy minimization and visualize again, we will see that the hexane chain and He atom will be spaced far apart, as being very close together (to the point of overlapping) would be unfavorable.

In [8]:
hexane.energy_minimize()

In [9]:
hexane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

If we instead perform the same operations but specify that we should ignore He via the `ignore_compounds` variable, the optimization will procedure such that the He atom is completely ignore and still sits at in the middle.

In [10]:
hexane = Alkane(n=6)

helium = mb.Compound(pos=hexane.labels['chain'].labels['monomer'][2].pos+[0.0,0.0,0.05], name='He', element='He')
hexane.add(helium)
hexane.energy_minimize(shift_com=False, ignore_compounds=[helium])
hexane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

We can simultaneously ignore a `Compound` and use it within a distance constraint. In such a case, the ignored compound will act as basically a reference point to constrain the optimization. 


Below, we will ignore the He atom and use it to specify distance constraint with the a terminal carbons, to create a ring-like structure.

In [11]:
hexane = Alkane(n=6)

helium = mb.Compound(pos=hexane.labels['chain'].labels['monomer'][2].pos+[0.0,0.0,0.05], name='He', element='He')
endpoint0 = hexane.labels['chain'].labels['CH3'][0].labels['C'][0]
endpoint1 = hexane.labels['chain'].labels['CH3'][1].labels['C'][0]


constraints = [[(helium, endpoint0), 0.1], 
               [(helium, endpoint1), 0.1],
              ]

hexane.add(helium)
hexane.energy_minimize(steps=10000,shift_com=False, ignore_compounds=helium, distance_constraints=constraints)


diff = np.linalg.norm(helium.pos-endpoint0.pos)
print("distance between endpoint0 and ignored helium: ", diff)

diff = np.linalg.norm(helium.pos-endpoint1.pos)
print("distance between endpoint1 and ignored helium: ", diff)

distance between endpoint0 and ignored helium:  0.10431424
distance between endpoint1 and ignored helium:  0.10431055


In [12]:
hexane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol